In [2]:
import os
import json
#wait 30 min before starting
import time

def filter_conditions(output_path, file):
    sims_of_interest = []

    if 'output_15sec_cand1_default' in output_path:
        sims_of_interest = ['gen_0_cand_0']
    elif 'output_inhib' in output_path:
        sims_of_interest = ['gen_1_cand_0', 'gen_3_cand_2', 'gen_4_cand_2']
    elif 'output_1' in output_path:
        sims_of_interest = ['gen_3_cand_3', 'gen_4_cand_5']

        #return False
    
    return not any(sim in file for sim in sims_of_interest)

try: from batch_run_files.plot_config import plot_sim_figs
except: from plot_config import plot_sim_figs
def plot_sims():
    plot_report = {}

    #print(f"Generation Complete. Plotting figures with below threshold fitness: {fitness_thresh}")
    #print(f'Plotting...{gen_folder}')        

    batch_run_path = output_path
    #gen_path = os.path.join(batch_run_path, gen_folder)
    plot_report_path = f'{output_path}/{gen_folder}/plot_report.json'
    net_activity_params = {'binSize': .03*1000, 'gaussianSigma': .12*1000, 'thresholdBurst': 1.0}
    plot_report = plot_sim_figs(gen_path, fresh_figs = fresh_figs, fitness_threshold = fitness_thresh, simLabel = None, net_activity_params = net_activity_params, timeout = timeout)
    # if len(plot_report) > 0:    
    #     break
    return plot_report
    

timeout = None
fitness_thresh = None
output_paths = [
                '/mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb7_Figure_Generation/selected_data/output_15sec_cand1_default', # default fig
                #'/mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb7_Figure_Generation/selected_data/15_sec_cand1_increasedK/output_k1',
                '/mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb7_Figure_Generation/selected_data/output_1', #excite
                '/mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb7_Figure_Generation/selected_data/output_inhib', #inhib
                # '/mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb7_Figure_Generation/selected_data/output_k1',
                # '/mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb7_Figure_Generation/selected_data/output_k2'
                ]
plot_switch = False
fresh_figs = True
filter_ = True
if plot_switch:
    for output_path in output_paths:
        print(f'{output_path}')
        gen_folders = os.listdir(output_path)
        #sort by generation number, last generation first
        gen_folders.sort(reverse=False)
        for gen_folder in os.listdir(output_path):
            if filter_ is True:
                #check if files is file
                if os.path.isfile(f'{output_path}/{gen_folder}') is True:
                    file = f'{output_path}/{gen_folder}'
                    continue_switch = filter_conditions(output_path, file)
                    if continue_switch is True: continue 
                #get all files in folder
                files = os.listdir(f'{output_path}/{gen_folder}')
                for file in files:
                    continue_switch = filter_conditions(output_path, file)
                    if continue_switch is False: 
                        gen_path = f'{output_path}/{gen_folder}/{file}/'
                        if '_data.json' not in file: continue
                        print(f'Plotting...{gen_path}')
                        plot_report = plot_sims()
                        print(plot_report)              
                if continue_switch is True:
                    continue
            else:
                gen_path = os.path.join(output_path, gen_folder)
                print(f'Plotting...{gen_path}')
                #plot_report = plot_sims()      
        

In [6]:
## Prepare Folder for Roy
import shutil
import os
import fnmatch

#Get current nb directory, and create a new folder for the reports
nb_dir = os.getcwd()
reports_path = os.path.join(nb_dir, 'reports')
overwrite = True
filter_ = True
#destination = 'reports' 
destination = 'reports_roy6'

#walk through output_path, find all .svg and .png files, copy to new folder
for output_path in output_paths:
    for root, dirs, files in os.walk(output_path):
        if '.archive' in root: continue
        for file in files:
            if filter_:
                if filter_conditions(root, file):
                    continue
            report_file_types = ['.svg', '.png', '.pdf', '.csv', '.run', '.err', '.json', '.log']
            image_file_types = ['.svg', '.png'] 
            info_file_types = ['.pdf', 
                            #'.csv', 
                            '.run', '.err', '.json', '.log'] 
            if file.endswith(tuple(report_file_types)):
                file_path = os.path.join(root, file)
                #new_path = file_path.replace(output_path, reports_path)
                new_path = file_path.replace('selected_data', destination)
                new_dir = os.path.dirname(new_path)

                if file.endswith(tuple(info_file_types)):
                # Split the file path into directories
                    directories = file_path.split(os.sep)
                    # Check if any directory matches the pattern "gen_*"
                    if any(fnmatch.fnmatch(dir, 'gen_*') for dir in directories):
                        continue  # Skip info files in gen folders
                    new_dir = os.path.join(new_dir, 'batch_run_files')
                    new_path = os.path.join(new_dir, file)
                
                #If file name contains "param_summary_row", copy to the grandparent folder instead
                if 'param_summary_row.png' in file:
                    new_dir = os.path.dirname(new_dir)
                    new_dir = os.path.dirname(new_dir)
                    new_path = os.path.join(new_dir, file)
                
                #If file name contains ".csv"
                if '.csv' in file:
                    #new_dir = os.path.dirname(new_dir)
                    new_path = os.path.join(new_dir, file)

                #make sure the new directory exists
                assert 'reports' in new_path
                if not os.path.exists(new_dir):
                    os.makedirs(new_dir)
                if not os.path.exists(new_path) and overwrite is False:
                    file_name = os.path.basename(file_path)
                    file_dir = os.path.basename(os.path.dirname(file_path))
                    try: assert file_dir in file_name
                    except: 
                        print(f'Error: {file_dir} not in {file_name}')
                        continue
                    shutil.copy(file_path, new_path)
                    print(f'Copied {file_path} to {new_path}')
                elif overwrite is True:
                    file_name = os.path.basename(file_path)
                    file_dir = os.path.basename(os.path.dirname(file_path))
                    try: assert file_dir in file_name
                    except: 
                        print(f'Error: {file_dir} not in {file_name}')
                        continue
                    shutil.copy(file_path, new_path)
                    print(f'Copied {file_path} to {new_path}')
                

Copied /mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb7_Figure_Generation/selected_data/output_15sec_cand1_default/gen_0/gen_0_cand_0/gen_0_cand_0__conn_matrix.svg to /mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb7_Figure_Generation/reports_roy6/output_15sec_cand1_default/gen_0/gen_0_cand_0/gen_0_cand_0__conn_matrix.svg
Copied /mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb7_Figure_Generation/selected_data/output_15sec_cand1_default/gen_0/gen_0_cand_0/gen_0_cand_0__raster.png to /mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb7_Figure_Generation/reports_roy6/output_15sec_cand1_default/gen_0/gen_0_cand_0/gen_0_cand_0__raster.png
Copied /mnt/disk15tb/adam/git_workspace/netpyne_2DNetworkSimulations/2DNet_simulations/BurstingPlotDevelopment/nb7_Figure_Genera